# Лабораторная работа №2. Дискретно-событийное моделирование на примере задачи вулканической баллистики

**Цель** работы - закрепить знания и получить опыт решения физической задачи путём компьютерного моделирования с применением дискретно-событийного подхода.

## Описание задачи

Смоделировать процесс разлёта камней из кратера вулкана при его извержении, получив в результате координаты точек падения камней на горизонтальную плоскость $z=0$.
Пространство трёхмерно, оси $Ox$ и $Oy$ лежат в горизонтальной плоскости, ось $Oz$ направлена вертикально вверх.

Используйте дискретно-событийный подход к моделированию извержения.
В этом может помочь библиотека [SimPy](https://simpy.readthedocs.io/en/latest/index.html).

Процесс извержения характеризуется следующими параметрами:

* $R_{\rm c}$ - радиус кратера;
* $D_{\rm c}$ - глубина кратера;
* $H > 0$ - высота кратера над горизонтальной плоскостью;
* $\mu_v$ и $\sigma_v$ - параметры нормального распределения _величины_ скорости камней: $v = \|\vec v\| \sim \mathcal{N}(\mu_v, \sigma_v)$, где $\vec v$ - вектор скорости камня;
* $\varphi \in [0, 2\pi]$ - диапазон разлёта бомб по углу азимута;
* $\mu_\theta$ и $\sigma_\theta$ - параметры нормального распределения угла возвышения (наклона к горизонту) вектора скорости камней: $\theta \sim \mathcal{N}(\mu_\theta, \sigma_\theta)$;
* $R_{\rm min}$ и $R_{\rm max}$ - минимальный и максимальный радиусы бомб. Распределение величины радиуса камня - равномерное;
* $\rho$ - плотность материала камней;
* $\beta_{\rm erup}$ - среднее время между выбросами камней при извержении.
  Случайная величина временного промежутка между выбросами $\Delta t_{\rm erup}$ распределена по экспоненциальному закону с параметром $\beta_{\rm erup}$.
  
При извержении новые _бомбы_ (камни) добавляются в список `flyings` (летящие).
При приземлении камень убирается из списка `flyings` и добавляется в список приземлившихся `fallens` (упавшие).

**Задачи** работы:

1. Создать компьютерную модель процесса извержения вулкана.
   Реализовать возможность моделирования с учётом и без учёта столкновения между камнями.
2. Составить отчёт по лабораторной работе (ЛР).
   Требования к отчёту см. во [введении](intro_report).
   Привести значения параметров (исходных данных и пр.) модели, полученные результаты в виде графика аналогичного графику, показанному на лекциях, а также проанализировать полученные результаты.

## Методические указания

### Модель камня (бомбы)

Бомба имеет форму шара радиуса $R$, массу $m$.
Её положение и движение описываются радиус-вектором $\vec{r}(t)$ и вектором скорости $\vec{v}(t)$ соответственно.

Класс камня может иметь флаг `is_collided`, указывающий на то, сталкивался ли камень (`True`) или нет (`False`).

### Модель извержения (разлёта камней)

Траектория каждого камня описывается законом движения свободно падающего тела:

$$
\begin{split}
\vec r(\tau)
    &= \vec r(0) + \vec v(0) \tau + \frac{\vec{g} \tau^2}{2} = \\
    &= \vec{r}_0 + \vec{v}_0 \tau + \frac{\vec{g} \tau^2}{2},
\end{split}
$$

где
$\tau$ - "собственное" время камня: $\tau = t - t_{\rm b}$ и $0 \le \tau \le t$;
$t$ - глобальное время: $t \ge 0$;
$t_{\rm b}$ - глобальное время извержения камня;
$\vec g = (0, 0, -9.81)$ м/с$^2$ - вектор ускорения свободного падения.

Этот же закон можно записать и в глобальном времени $t$:

$$
\vec r(t) =
    \vec{r}_0 + \vec{v}_0 \tau(t) + \frac{\vec{g} \tau^2(t)}{2},
$$

### Извержение

Процесс извержения характеризуется

* средним временем между выбросами $\beta_{\rm erup}$ и
* числом $N > 0$ выбрасываемых при этом камней.

Случайная величина времени между двумя соседними выбросами $\Delta t_{\rm erup}$ распределена по экспоненциальному закону с функцией плотности вероятности

$$
\Delta t_{\rm erup} \sim
    \cfrac{1}{\beta_{\rm erup}} \exp{\left( -\cfrac{\Delta t_{\rm erup}}{\beta_{\rm erup}} \right)}.
$$

При выбросе (событие `ERUPTION`):

1. Генерируется $N$ новых бомб.
2. Рассчитывается абсолютное время $t_{\rm g}$ их падения на землю, т.е. достижение высоты $z=0$.
3. Для каждой новой бомбы рассчитывается ближайший момент времени $t_{\rm col}$, когда она столкнётся с любой бомбой из списка `flyings`.
4. После этого новая бомба добавляется в список `flyings`.

### Столкновение

При столкновении двух бомб $B_1$ и $B_2$ (событие `COLLISION`):

1. Очищается очередь событий, связанных с бомбами $B_1$ и $B_2$.
2. Обновляются векторы начальных скоростей $\vec v^{B_1}_0$ и $\vec v^{B_2}_0$ и радиус-векторы начального положения $\vec r^{B_1}_0$ и $\vec r^{B_2}_0$ бомб $B_1$ и $B_2$.
3. Пересчитываются времена падения $t^{B_1}_{\rm g}$ и $t^{B_2}_{\rm g}$ бомб $B_1$ и $B_2$.
4. Пересчитываются моменты времени столкновения $t^{B_1}_{\rm col}$ и $t^{B_2}_{\rm col}$ бомб $B_1$ и $B_2$ с другими камнями из списка `flyings`.

#### Момент столкновения камней

Камни $B_1$ и $B_2$ сталкиваются, если расстояние между их центрами равно сумме их радиусов $R_1$ и $R_2$, т.е.

$$
\| \vec r_1(t) - \vec r_2(t) \| = R_1 + R_2.
$$

При численном решении данного уравнения определяется момент столкновения камней $t_{\rm col}$.
Способы численного решения уравнений в Python можно найти в [справочнике](https://unexpectedcoder.github.io/sm6-py-cookbook/intro.html).

#### Расчёт скоростей после столкновения

Скорость сближения (*closing*) тел в момент удара

$$
v_{\rm c} =
    -\left[ \vec{v}_1(t_{\rm col}) - \vec{v}_2(t_{\rm col}) \right]
    \cdot \vec{n},
$$

где $\vec{n}$ - вектор единичной нормали при столкновении:

$$
\vec n =
    \cfrac
    {\vec r_1(t_{\rm col}) - \vec r_2(t_{\rm col})}
    {\| \vec r_1(t_{\rm col}) - \vec r_2(t_{\rm col}) \|}.
$$

Скорость разлёта (*separating*) тел до столкновения

$$v_{\rm s} = -v_{\rm c}$$

и после него

$$
\vec v_{\rm s}' = -c v_{\rm s},
$$

где $c$ - коэффициент восстановления: $0 \le c \le 1$.

Изменение скорости при ударе:

$$
\Delta v_{\rm s} = v_{\rm s}' - v_{\rm s}.
$$

Вектор полного импульса:

$$\vec p = \cfrac{m_1 m_2}{m_1 + m_2} \Delta v_{\rm s} \vec n.$$

Следовательно, исходя из закона сохранения импульса, возможно рассчитать скорость каждого тела после столкновения:

$$\vec v_1' = \vec v_1 + \cfrac{\vec p}{m_1},$$

$$\vec v_2' = \vec v_2 - \cfrac{\vec p}{m_2}.$$

### Приземление

При приземлении камня (событие `GROUND`):

1. Очищается очередь событий, связанных с этим камнем.
2. Камень переносится из списка `flyings` в список `fallens`.

В конце симуляции координаты упавших камней берутся из массива `fallens` и отображаются на графике, а именно, на виде сверху.
Начало системы отчёта располагается в центре кратера.

## Шаблон кода

Библиотеки, которые могут пригодиться:

```{code}
from numpy.random import Generator, PCG64
from scipy.optimize import root_scalar
from typing import Dict, List
import matplotlib.pyplot as plt
import numpy as np
import simpy as sim
```

Класс бомбы:

```{code}
class Bomb:
    def __init__(self,
                 t_erup: float,
                 r0: np.ndarray,
                 v0: np.ndarray,
                 mass: float,
                 radius: float):
        self.t_erup = t_erup
        self.r = r0
        self.v = v0
        self.m = mass
        self.R = radius
        # TODO: + ваши поля, которые посчитаете нужным использовать
    
    def calc_r(self, t: float):
        """Рассчитать радиус-вектор в момент времени `t`."""
        pass
    
    def calc_v(self, t: float):
        """Рассчитать вектор скорости в момент времени `t`."""
        pass
    
    def is_collided(self):
        """Столкивался ли камень."""
        pass
    
    def xy_fall(self):
        """Координаты падения."""
        pass
```

Глобальные списки летящих камней и камней на земле, а также словарь (хэш-таблица) процессов каждого экземпляра бомбы:

```{code}
flyings, fallens = [], []
processes: Dict[Bomb, List[sim.Process]] = {}
```

Запись `processes: Dict[Bomb, List[sim.Process]]` означает, что мы объявляем словарь, ключи которого имеют тип `Bomb`, а значения - тип `sim.Process` (процесс в SimPy).

```{note}
Ключи для словарей должны быть _хешируемыми_, т.е. уникальными.
В классе `Bomb` нет реализации функции `__hash__`, но она и не нужна.
За уникальность экземпляров `Bomb` отвечает адрес памяти, в которой каждый конкретный экземпляр хранится.
```

Генератор случайных чисел инициализируется, например, так:

```{code}
rs = Generator(PCG64(seed=2103))
```

_Функция-процесс_ выброса камней при извержении:

```{code}
def eruption(env: sim.Environment, dt: float, n: int, rs: Generator):
    """Процесс (в терминах SimPy) выброса камней.
    
    env :
        Объект среды SimPy,
        отвечающий за управление и обработку событий.
    dt :
        Время между выбросами.
    n :
        Число выбрасываемых камней.
    rs :
        Генератор чисел.
    """
    # Ожидание события ERUPTION...
    yield env.timeout(dt)
    # и вот оно наступило.
    # TODO: действия при наступлении события ERUPTION
    ...
```

Всего таких процессов при моделировании $N_{\rm erups}$.

Функция генерации камней:

```{code}
def gen_bombs(env: sim.Environment, n: int, rs: Generator):
    """Сгенерировать n бомб (камней).

    env :
        Объект среды SimPy,
        отвечающий за управление и обработку событий.
    n :
        Число выбрасываемых камней.
    rs :
        Генератор чисел.
    """
    pass
```

_Функция-процесс_, отвечающая действию при наступлении события `GROUND`:

```{code}
def ground(env: sim.Environment, dt: float, b: Bomb):
    """Процесс (в терминах SimPy), происходящий при наступлении события GROUND.
    
    env :
        Объект среды SimPy,
        отвечающий за управление и обработку событий.
    dt :
        Задержка начала процесса.
    b :
        Бомба (камень).
    """
    try:
        yield env.timeout(dt)
        # TODO: ...
    except sim.Interrupt:
        return
```

```{important}
Блок `try...except` в функциях-процессах необходим для возможности прерывания (_interrupt_) процесса, если возможность прерывания требуется.
```

_Функция-процесс_, описывающая действие при наступлении события `COLLISION`:

```{code}
def collision(env: sim.Environment, dt: float, b1: Bomb, b2: Bomb):
    """Процесс (в терминах SimPy), происходящий при наступлении события COLLISION.
    
    env :
        Объект среды SimPy,
        отвечающий за управление и обработку событий.
    dt :
        Задержка начала процесса.
    b1 и b2 :
        Бомбы.
    """
    try:
        yield env.timeout(dt)
        # TODO: действия при наступлении события столкновения
        ...
    except sim.Interrupt:
        return
```

Функция очистки очереди событий, связанных с бомбой `b`:

```{code}
def clear_queue(b: Bomb):
    for proc in processes[b]:
        try:
            proc.interrupt()
        except RuntimeError:
            continue
```

Функция расчёта времени падения камня на землю ($z=0$):

```{code}
def when_ground(b: Bomb):
    pass
```

Функция расчёта скоростей бомб `b1` и `b2` после их столкновения в момент времени `t`:

```{code}
def calc_collision(t: float, b1: Bomb, b2: Bomb):
    pass
```

Функция расчёта момента времени столкновения $t_{\rm col}$ камня `b1` с камнем `b2`:

```{code}
def when_collision(b1: Bomb, b2: Bomb):
    pass
```

_Функция-процесс_ моделирования извержения:

```{code}
def simulate(env: sim.Environment,
             n_erups: int,
             allowed_collisions: bool):
    """Процесс моделирования.
    
    env :
        Объект среды SimPy,
        отвечающий за управление и обработку событий.
    n_erups :
        Число выбросов.
    allowed_collisions :
        Столкновения учитываются (True) или нет (False).
    """
    pass
```

Экземпляр среды SimPy **должен создаваться единожды** и передаваться во все функции, в которых он требуется.
Инициализация среды происходит просто:

```{code}
env = sim.Environment()
```

Таким образом, создав экземпляр `sim.Environment`, вы вызываете функцию `simulate(...)` с заданными вами параметрами.
Она в свою очередь инициализирует все необходимые процессы и запускает симуляцию внутри SimPy путём вызова `env.run()`.

### Визуализация

Универсальная функция визуализации точек падения камней:

```{code}
def top_view(xy: np.ndarray, ax=None, **kw):
    if ax is None:
        _, ax = plt.subplots()
    marker = kw.get("marker", ".")
    color = kw.get("color", "k")
    label = kw.get("label", "")
    alpha = kw.get("alpha", 1.)
    ax.plot(xy[:, 0], xy[:, 1],
            ls="", marker=marker, c=color, alpha=alpha, label=label)
    return ax
```

Пример итогового графика показан на рисунке ниже:

![Пример результата](./pics-des-vulkano/result.png)

## Рекомендации

* Используйте функции [`save`](https://numpy.org/doc/stable/reference/generated/numpy.save.html) и/или [`savez`](https://numpy.org/doc/stable/reference/generated/numpy.savez.html) библиотеки NumPy для сохранения точек падения.
  В этом случае вам не нужно будет хранить все полученные данные в оперативной памяти.
  После моделирования вы сможете читать данные в другой части кода, отделённой от той его части, которая отвечает за моделирование.
* Решить уравнение встречи бомб можете численным методом [`root_scalar`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.root_scalar.html) из библиотеки SciPy.
* Используйте библиотеку дискретно-событийного моделирования [`SimPy`](https://simpy.readthedocs.io/en/latest/), чтобы не изобретать велосипед.
  Либо реализуйте программу согласно псевдокоду с лекции по теме дискретно-событийного моделирования.

## См. также

1. Библиотека фреймворка дискретно-событийного моделирования [SimPy](https://simpy.readthedocs.io/en/latest/index.html).
2. Численное решение уравнений можно найти в [справочнике](https://unexpectedcoder.github.io/sm6-py-cookbook/intro.html).
   Там же содержится информация по построению графиков.
3. Требования к отчёту см. в разделе "Актуальная информация" в [группе кафедры](https://vk.com/bmstu_sm6) и во [введении](intro_report).